# Cálculo de porcentajes de uso del suelo

Este script abarca el flujo de trabajo completo para obtener una tabla con el porcentaje de ocupación de uso del suelo para cada una de las ciudades de más de 100.000 habitantes de España, excluyendo el uso sin edificar y otros. Los pasos que contempla este flujo de trabajo son los siguientes: 

1.- Crear nueva columna para la categoría uso agregado.
2.- Reclasificar los usos a una nueva clase uso agregado.
3.- Asignar un nuevo código numérico a los usos de la categoría uso agregado.
4.- Exportar los shapefile con las modificaciones realizadas.
5.- Eliminar las clases sin edificar y otros.
6.- Calcular los porcentajes de ocupación de cada uso respecto al total, sin considerar sin edificar y otros. 

In [30]:
import pandas as pd
import geopandas as gpd
import os
import glob
import string

 ### Incorporar una barra de progreso por medio de la libraría tqdm

In [13]:
from tqdm import tqdm, trange 
from time import sleep
from tqdm.notebook import tqdm

### Definir el directorio utilizando la librería os


In [12]:
path =(r"F:/Respaldo toshiba/projectos/shapes/marcos_clasificacion_ag/")
# Definir directorio de trabajo
os.chdir(path)

Leer de forma iterativa cada uno de los shapefiles y almacenarlos en un diccionario con clave igual al nombre del archivo, y valor correspondiente al shapefile.

In [33]:
# Crear un diccionario vacío
dict_shapes = {}
# Barra de progreso indicando el número total de elementos
pbar = tqdm(total = 72) 
# IIterar sobre cada uno de los archivos del directorio 
for file in os.listdir():
   if file.endswith('.shp') and file[0].isalpha():
       temp = gpd.read_file(file)
       dict_shapes[file] = temp
       pbar.update(1)
       sleep(0.0001)
pbar.close()

  0%|          | 0/72 [00:00<?, ?it/s]

Crear un nuevo diccionario: dict_areas, con los shapefiles modificados, incluyendo las clases agregadas para cada geometria. Este atributo no viene incluido en el shapefile original.

In [35]:
dict_areas = {}
for key, value in dict_shapes.items():
    temps = dict_shapes[key].copy()
    temps['Area'] = temps.area
    dict_areas[key] = temps

1. Agregar clases usando códigos comunes entre categorías.

Leer archivos csv con las categorías agregadas

In [36]:
clases_agregadas = pd.read_csv(r"F:/Respaldo toshiba/projectos/shapes/clases_agregadas.csv")
clases_agregadas

,USO,USO_AG
0,COM,COM
1,ED_SING,ED_SING
2,EQUIP_EDU,EQUIP
3,EQUIP_OTR,EQUIP
4,EQUIP_SANI,EQUIP
5,HOS_REST,OCIO
6,IND,IND
7,IND_MX,IND
8,OCIO_ESP,OCIO
9,OFI,OFI


Definimos una función para hacer una join usando la función merge de pandas. Ejecuta un leftjoin a partir de la columna común uso.

In [37]:
def uso_agregado(data_in, clases):
    data_out = {}
    for key, value in data_in.items():
        data_out[key] = value.merge(clases, on = 'USO', how = 'left')
    return data_out


Aplicar la función y obtenemos un nuevo diccionario con el uso agregado

In [38]:
uso_agregado = uso_agregado(dict_areas,clases_agregadas)

Definimos una función para asignar un codigo por categoria agregada

In [54]:
def asignacion_codigo_categorias(data_in):
    data_out = {}
    for key, df in data_in.items():
        df = df.reset_index()
        usos = df['USO_AG'].unique()
        cod_usos = np.arange(len(usos))
        cod_map = pd.Series(cod_usos, index=usos)
        df['cod_usos_a'] = df['USO_AG'].map(cod_map)
        data_out[key] = df
    return data_out

Aplicamos la función para obtener un código númerico asociado a cada categoría.

In [2]:
uso_agregado_cat = asignacion_codigo_categorias(uso_agregado)

In [ ]:
Calcular la suma de las áreas de cada categoría agregada

In [8]:
df_final = pd.DataFrame()
for key, value in uso_agregado_cat.items():
    temp = uso_agregado_cat[key] 
    temp_cat = temp.groupby('USO_AG')['Area'].sum()
    temp_area = temp.groupby('USO_AG')['Area'].sum().sum()
    temp_porcentaje = (temp_cat/temp_area)*100
    df_temp = temp_porcentaje.to_frame().rename(columns = {'Area':key})
    df_final = pd.concat([df_final,df_temp], axis = 1)

### Exportar los datos a un csv

Definimos una función para eliminar la categoría otros de todos los shapesfiles.

In [10]:
dict_filter_sin_otros = {}
for key, value in uso_agregado_gtp_cat.items():
    ciudad = uso_agregado_gtp_cat[key].copy()
    ciudad1 = ciudad[ciudad.loc[:,'USO_AG'] != "OTROS"].copy()
    dict_filter_sin_otros[key] = ciudad1

Calcular los porcentajes sin considerar la categoría otros. 

In [11]:
df_sin_otros = pd.DataFrame()
for key, value in dict_filter_otros.items():
    temp = dict_filter_otros[key] 
    temp_cat = temp.groupby('USO_AG')['Area'].sum()
    temp_area = temp.groupby('USO_AG')['Area'].sum().sum()
    temp_porcentaje = (temp_cat/temp_area)*100
    df_temp = temp_porcentaje.to_frame().rename(columns = {'Area':key})
    df_otros = pd.concat([df_sin_otros,df_temp], axis = 1)

Finalmente, volvemos a calcular los porcentajes de cada categoría respecto al total, pero sin considerar la categoría otros, y sin edificación.

In [ ]:
dict_filter = {}
for key, value in uso_agregado_gtp_cat.items():
    ciudad = uso_agregado_gtp_cat[key].copy()
    ciudad1 = ciudad[ciudad.loc[:,'USO_AG'] != "SIN_EDIF"].copy()
    ciudad2 = ciudad1[ciudad1.loc[:,'USO_AG'] != "OTROS"].copy()
    dict_filter[key] = ciudad2

In [ ]:
df_final_filter = pd.DataFrame()
for key, value in dict_filter.items():
    temp = dict_filter[key] 
    temp_cat = temp.groupby('USO_AG')['Area'].sum()
    temp_area = temp.groupby('USO_AG')['Area'].sum().sum()
    temp_porcentaje = (temp_cat/temp_area)*100
    df_temp = temp_porcentaje.to_frame().rename(columns = {'Area':key})
    df_final_filter = pd.concat([df_final_filter,df_temp], axis = 1)

Exportar la tabla final

In [ ]:
df_final_filter.to_csv('')

La tabla tiene las variables en las filas y las observaciones en las columnas, por lo tanto tenemos que transformarla (Formato wide a long)

In [ ]:
df_final_filter_melt = df_final_filter.melt(id_vars = "USO_AG")

In [ ]:
df_final_filter_pivot = df_final_filter_melt.pivot(columns = "USO_AG", values = 'value', index = 'variable')

In [ ]:
df_final_filter_pivot = df_final_filter_pivot.reset_index()

In [ ]:
df_final_filter_pivot.to_csv("")

### Finalmente queremos corregir el nombre de las ciudades para dejar solamente el nombre de las ciudades. Esto lo haremos utilizando expresiones regulares.

In [ ]:
df_final_filter_pivot['Ciudad'] = df_final_filter_pivot['Ciudades'].str.replace('_', ' ')
df_final_filter_pivot['Ciudad'] = df_final_filter_pivot['Ciudad'].str.replace(r'\.txt$', '')
df_final_filter_pivot['Ciudad'] = df_final_filter_pivot['Ciudad'].apply(lambda x: x.capitalize())

In [ ]:
rep = {'Alcorcon':'Alcorcón','Ciudad real':'Ciudad Real','Bodajoz':'Badajoz','Dos hermanas':'Dos Hermanas','Fuentelabra':'Fuenlabrada',
'Leganes':'Leganés','Logronio':'Logroño','Mataro':'Mataró','Mostoles':'Móstoles','Santa coloma':'Santa Coloma','Santa cruz tenerife':'Santa Cruz de Tenerife',
'San cristobal':'San Cristóbal','Terrasa':'Terrassa','Terragona':'Tarragona','Corunia':'Coruña','Gijon':'Gijón',
'Las palmas':'Palmas de Gran Canaria, Las'}

In [ ]:
def replace_cities(df, replacements,col):
    for city, replacement in replacements.items():
        df[col] = df[col].str.replace(city, replacement)
    return df

In [2]:
df_final_filter_pivot_replace = replace_cities(df_final_filter_pivot, rep,'Ciudad')